In [1]:
import  json
from os import listdir
from os.path import isfile, join
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

Load the data from JSON

In [ ]:
data = json.load(open('rumoureval-subtaskA-train.json'))

dev = json.load(open('rumoureval-subtaskA-dev.json'))



Match the respective tweets by ID

In [ ]:
csv = open('traindata.csv', "w", encoding='utf-8-sig')
columnTitleRow = "ID,tweet_text,results\n"
csv.write(columnTitleRow)

for k,v in data.items():
    json_path=""
    mypath="C:/Users/Nicole/semeval2017-task8-dataset/rumoureval-data/"
    
    for d in listdir(mypath):
        for thread in listdir(mypath+d):
            if k == thread:
                json_path=mypath+d+"/"+thread+"/source-tweet/"+k+".json"
                break
            else:                
                reply_path=mypath+d+"/"+thread+"/replies/"
                if k+".json" in listdir(reply_path):
                    json_path=reply_path+k+".json"
                    break
        if json_path!="":
            break
    if (isfile(json_path)):
            source = json.load(open(json_path, 'r'))
            print(json_path)
    else:
        print ("False:" + k)
        print(json_path)
        print("end\n")
    #print (source)
    reply_tweet = source['text']
    reply_tweet = reply_tweet.replace("\n", " ")
    reply_tweet = reply_tweet.replace("\"", "\"\"")
    row = k + "," + "\"" + reply_tweet + "\"" + "," + v + "\n"
    csv.write(row)
    #print (source['text'])
    #print (v+"\n")
    

In [3]:
dataset = pd.read_csv('traindata.csv', error_bad_lines=False)

In [ ]:
dataset.head()

In [ ]:
csv.close()

Information of the training data

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 3 columns):
ID             4238 non-null int64
 tweet_text    4238 non-null object
 results       4238 non-null object
dtypes: int64(1), object(2)
memory usage: 99.4+ KB


use the following code to see if the data is loaded right

In [ ]:
dataset[dataset.isnull().any(axis=1)]

In [8]:
np.sum(dataset.isnull().any(axis=1))

0

In [ ]:
dataset.isnull().any(axis=0)

In [9]:
dataset = dataset.rename(columns={" tweet_text": "tweet_text", " results": "results"})

the tweets of different classes

In [46]:
dataset['results'].value_counts()

comment    2734
support     841
deny        333
query       330
Name: results, dtype: int64

The data is skew

we will need to try oversampling the 3 groups other than 'comment' group, devide them into 4 groups

In [ ]:
dfcomment = dataset[dataset.results == 'comment']

In [ ]:
dfsupport = dataset[dataset.results == 'support']

In [ ]:
dfdeny = dataset[dataset.results == 'deny']

In [ ]:
dfquery = dataset[dataset.results == 'query']

In [ ]:
dfquery.head()

# data cleansing

In [ ]:
dataset['text_len'] = [len(t) for t in dataset.tweet_text]

In [ ]:
dataset.head()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
plt.boxplot(dataset.text_len)
plt.show()

In [ ]:
testing = dataset[dataset['text_len']>140]

In [ ]:
testing.head()

tweet has length limit with 140, to see why they exceed 140

In [ ]:
testing['tweet_text'].iloc[1]

there are some HTML decoding problem

In [ ]:
len(testing['tweet_text'])

# HTML decoding

use the beautiful soup to do the html decoding

In [10]:
from bs4 import BeautifulSoup

run a test on it

In [ ]:
example1 = BeautifulSoup(testing['tweet_text'].iloc[3], 'html.parser')
example1.get_text()

In [ ]:
example1

In [ ]:
# def tweet_cleaner(text):
#     example1 = BeautifulSoup(text, 'html.parser')
#     return example1.get_text()
    

In [11]:
clean_tweet_texts = []
for i in dataset['tweet_text']:
    clean_tweet_texts.append(BeautifulSoup(i, 'html.parser').get_text())

In [12]:
len(clean_tweet_texts)

4238

In [13]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['results'] = dataset['results']
clean_df['ID'] = dataset['ID']
clean_df.head()

,text,results,ID
0,@max843 @DanteB4u @Alasscan_ This shows Micha...,comment,501760642928635904
1,@USATODAY Get ready to tweet the address #Spik...,comment,500270212198174720
2,@rikermacdaniels @CBCAlerts @CBCNews @JustinTr...,comment,524971210275565568
3,@RT_com Just kill them.,comment,552836882770690049
4,"@TheAnonMessage2 no, because that would requir...",support,500289931097296897


In [18]:
htmldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 3 columns):
text       4238 non-null object
results    4238 non-null object
ID         4238 non-null int64
dtypes: int64(1), object(2)
memory usage: 99.4+ KB


In [ ]:
clean_df.to_csv('training_data_with_html_decode.csv', index = None)

In [ ]:
htmldata = pd.read_csv('training_data_with_html_decode.csv')

In [17]:
htmldata

check the length again

In [ ]:
htmldata['text_len'] = [len(t) for t in htmldata.text]

In [ ]:
htmldata[htmldata['text_len']>140]

it's empty so we're good to go

In [ ]:
htmldata['text'].iloc[0]

In [ ]:
htmldata['text'].iloc[4217]

In [ ]:
htmldata['text'].iloc[9]

for pre-processing, there's a Python library which is used to deal with the tweets preprocessing, it will clean out:
URLs
Hashtags
Mentions

Emojis
Smileys

I decided to remove the URLs, Mentions,  Emojis, Smileys, numbers, but for hashtags, I will only remove the hashtag symbol and keep the word following the hashtag symbol as some of it is the part of the sentense and it's useful, so I'm not using the hashtags option here, and I will remove the Reserved words (RT, FAV) in the tweets

In [15]:
import preprocessor as p

In [16]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.NUMBER, p.OPT.SMILEY)

In [ ]:
re.sub(' RT| FAV', ' ', p.clean('Preprocessor is #awesome 👍 https://github.com/s/preprocessor RTl'))

make small tests first

In [ ]:
p.clean(htmldata['text'].iloc[0])

In [ ]:
re.sub(' RT', ' ', p.clean(htmldata['text'].iloc[0]))

In [ ]:
htmldata['text'].iloc[4217]

In [ ]:
p.clean(htmldata['text'].iloc[4217])

remove punctuations, non-alphabaet characters and separates the word with a single space

In [ ]:
' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",testtext).split())

do it on all the data

In [19]:
clean_tweet_texts1 = []
for i in htmldata['text']:
    clean_tweet_texts1.append(re.sub(' RT', ' ',p.clean(i)))

In [20]:
clean_tweet_texts2 = []
for i in clean_tweet_texts1:
    clean_tweet_texts2.append(' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",i).split()))

In [21]:
clean_tweet_texts2_df = pd.DataFrame(clean_tweet_texts2,columns=['text'])
clean_tweet_texts2_df['results'] = htmldata['results']
clean_tweet_texts2_df['ID'] = htmldata['ID']
clean_tweet_texts2_df.head()

,text,results,ID
0,This shows Michael paid for cigarillos No robb...,comment,501760642928635904
1,Get ready to tweet the address SpikeLee and ot...,comment,500270212198174720
2,your radical Muslims are being funded by Ameri...,comment,524971210275565568
3,Just kill them,comment,552836882770690049
4,no because that would require reasoning and th...,support,500289931097296897


In [ ]:
clean_tweet_texts2_df['text'].iloc[0]

In [30]:
clean_tweet_texts2_df.to_csv('clean_trainingdata.csv', index = None)

In [31]:
df = pd.read_csv('clean_trainingdata.csv')

In [23]:
df = clean_tweet_texts2_df

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 3 columns):
text       4136 non-null object
results    4238 non-null object
ID         4238 non-null int64
dtypes: int64(1), object(2)
memory usage: 99.4+ KB


In [33]:
df[df.isnull().any(axis=1)]

,text,results,ID
9,NaN,comment,552811591683821568
29,NaN,comment,544520601038098433
62,NaN,comment,524937817924067328
64,NaN,query,553591557325336576
95,NaN,comment,544518918912495616
112,NaN,comment,553535342049902592
150,NaN,comment,524933596113801218
281,NaN,comment,544290755624198144
375,NaN,comment,544349387741495296
380,NaN,comment,544520377162952704


In [38]:
dataset.iloc[62]

ID            524937817924067328
tweet_text         @globeandmail
results                  comment
Name: 62, dtype: object

remove those empty texts

In [39]:
df.dropna(inplace=True)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4136 entries, 0 to 4237
Data columns (total 3 columns):
text       4136 non-null object
results    4136 non-null object
ID         4136 non-null int64
dtypes: int64(1), object(2)
memory usage: 129.2+ KB


In [44]:
df.to_csv('trainingdataset.csv', index = None)

In [45]:
df['results'].value_counts()

comment    2660
support     824
deny        328
query       324
Name: results, dtype: int64

In [47]:
dfcomment = df[df.results == 'comment']
dfsupport = df[df.results == 'support']
dfdeny = df[df.results == 'deny']
dfquery = df[df.results == 'query']

In [65]:
newdf2 = pd.concat([dfsupport, dfsupport])

In [68]:
len(newdf2)

2472

In [55]:
newdf = pd.concat([newdf, newdf]) #oversample the deny

In [60]:
newdf1 = pd.concat([newdf1, newdf1]) #oversample the query

In [67]:
newdf2 = pd.concat([newdf2, dfsupport]) #oversample the support

In [69]:
newdataset = pd.concat([newdf2, dfcomment,newdf,newdf1])

In [71]:
newdataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10348 entries, 4 to 4202
Data columns (total 3 columns):
text       10348 non-null object
results    10348 non-null object
ID         10348 non-null int64
dtypes: int64(1), object(2)
memory usage: 323.4+ KB


In [72]:
newdataset.to_csv('new_trainingdata.csv', index = None)